In [ ]:
require '../boast/LibconvBase.rb'

In [ ]:
ndims=NArray.int(3)
ndims[0]=62
ndims[1]=66
ndims[2]=65
ndimsmf=NArray.int(3)
ndimsmf=2*ndims

scal=NArray.float(3)
hgrid = NArray.float(3)
hgrid[0] = 0.5
hgrid[1] = 0.6
hgrid[2] = 0.7
(0..2).each{ |ind| scal[ind] = -0.5 / (hgrid[ind]*hgrid[ind])   }

In [ ]:
input_s1   = dataspace( 2*ndims,random=true)
data_p = dataspace(2*ndims)
work_p = dataspace(2*ndims) #may work well in s1,s0,r
data2_p = dataspace(2*ndims)
5

In [ ]:
dwt=generate_dwt()
iwt=generate_iwt()
mf=generate_mf()
imf=generate_imf()
s1r=generate_s1tor()
rs1=generate_rtos1()

In [ ]:
  suffix = ".c" if BOAST::get_lang == BOAST::C
  suffix = ".f90" if BOAST::get_lang == BOAST::FORTRAN
  File::open("poisson#{suffix}","w") { |f|
    f.puts k2
    f.puts k3
    f.puts k4
    f.puts k5
    f.puts k6
    f.puts k7
  }


In [ ]:
puts dwt.kernel.procedure.name

In [ ]:
#rs1f=generate_rtos1(nil,periodic=false)

In [ ]:
iwt.run(0, ndims, BC::PERIODIC, input_s1, work_p, 1.0,0.0)
dwt.run(0, ndims, BC::PERIODIC, work_p, data_p, 1.0,0.0)

In [ ]:
allclose(input_s1,data_p)

In [ ]:
iwt.run(0, ndims, BC::PERIODIC, input_s1, work_p, 1.0,0.0)
mf.run(0, ndimsmf, BC::PERIODIC, work_p, data_p, 1.0,0.0,0.0)
s1r.run(0, ndims, BC::PERIODIC, input_s1, data2_p, 1.0,0.0)

In [ ]:
allclose(data2_p,data_p)

In [ ]:
imf.run(0, ndimsmf, BC::PERIODIC, input_s1, work_p, 1.0,0.0,0.0)
dwt.run(0, ndims, BC::PERIODIC, work_p, data_p, 1.0,0.0)
rs1.run(0, ndims, BC::PERIODIC, input_s1, data2_p, 1.0,0.0)

In [ ]:
allclose(data2_p,data_p)

In [ ]:
ndims0=get_ld(0,ndims,BC::GROW,:iwt)
ndimsr=get_ld(0,2*ndims0,BC::GROW,:mf)
ndimsr1=2*get_ld(0,ndims,BC::GROW,:s1tor)
5

In [ ]:
data_s0=dataspace(2*ndims0)
data_sr=dataspace(ndimsr)
data_sr1=dataspace(ndimsr1)
5

In [ ]:
iwt.run(0, ndims, BC::GROW, input_s1, data_s0, 1.0,0.0)
mf.run(0, 2*ndims0, BC::GROW, data_s0, data_sr, 1.0,0.0,0.0)

In [ ]:
s1r.run(0, ndims, BC::GROW, input_s1, data_sr1, 1.0,0.0)

In [ ]:
#combine operations and verify result
p data_sr1[0..5,1,0]
p data_sr[0..5,1,0]
p data_sr1[-1,0,0]
5

In [ ]:
imf.run(0,2*ndims0,BC::SHRINK,data_sr,data_s0,1.0,0.0,0.0)
dwt.run(0,ndims,BC::SHRINK,data_s0,data_p,1.0,0.0)

In [ ]:
rs1.run(0,ndims,BC::SHRINK,data_sr1,data2_p,1.0,0.0)

In [ ]:
allclose(data_p,data2_p)

In [ ]:
#example of conversion from reals space into a multi-resolution
psi=Wavefunction::new(ndims,:r)
out=psi.rs1()
#take only the scaling function coefficients
out.select_s0()


In [ ]:
class DataSpace
  attr_reader :ndim
  attr_reader :ld
  def initialize(ndim,space) #bc=BC::PERIODIC,random=false,fill=nil)
    @ndim=[ndim].flatten
    @ld=options
  end
end

In [ ]:
module S1
  @s1s0=generate_dwt()
  @s1r=generate_s1tor()
  def self.s1s0
    return @s1s0
  end
  def self.s1r
    return @s1r
  end
  def s1s0(idim,options={})
    output=options.fetch(:output,nil)
    ldin=S1::s1s0.in_lds(@system.ndims[idim],@lds,@system.bc[idim])
    ldout=S1::s1s0.out_lds(@system.ndims[idim],@lds,@system.bc[idim])
    if not output then
      spaces=@spaces.dup
      spaces[idim]= :s0
      data=dataspace(2*lds)
      output=Wavefunction::new(@system,spaces: spaces, dataspace: data)
    else
      #here all the checks
      #ld of the input
      assert_compatibility_between 2*ldin, @lds
      #ld of the output
      assert_compatibility_between 2*ldout, output.lds
        #ld of the output a
    end
    S1::s1s0.run(idim,@ndims,@bc[idim], @dataspace, output, 1.0,0.0)
  end
    def assert_compatibility_between(lds,wfnlds)
    if lds.zip(wfnlds).any? {|e| e[0]>e[1] or wfnlds.odd?} then
      raise "Incompatible dimensions #{lds.join(',')}: #{wfnlds.join(',')}"
    end   
  end
end
  

In [ ]:
class System
  attr_reader :reference_space
  attr_reader :reference_dimensions
  attr_reader :boundary_conditions
  attr_reader :grow_direction
  def initialize(ndims,reference_space,bc,options={})
    @reference_dimensions=ndims
    @reference_space=reference_space
    @boundary_conditions=bc
    @grow_direction=options.fetch(:grow_direction,:s1s0r)    
  end
  
  def minimum_buffer_size(idim,op)
    return get_ld(idim,@ndims,@bc,op)
  end
end

In [ ]:
class Wavefunction
  attr_accessor :dataspace
  attr_reader :system
  attr_reader :present_spaces
  attr_reader :lds
  def initialize(system,options={})
    @system=system
    @present_spaces=options.fetch(:spaces,[system.reference_space]*ndims.length)
    @dataspace=options.fetch(:dataspace,nil)
    @lds=options.fetch(:lds,@dataspace ? @dataspace.shape : nil)
  end
end

In [ ]:
V3=System::new(ndims,reference_space= :s1)
psi=dataspace(ld=V3.ld*2^4)
V0=System::new(ndims,reference_space= :r,bc:free, tr = Transition::new(:r, :s1, :shrink ))
psi0r=Wavefunction::new(system=V0, :r)

#V1=V0.transformed_system(:s1)
#V1=System::new(ndims/2,reference_space= :s1,bc:free, Transition::new(:s1, :r, :grow ))
#psis1 = Wavefunction::new(system=V1) 

psi0s1 = Wavefunction::new(system=V0, :s1)
psi0s1 = psi0s0.rs1(:output => psi0s1)
V1 System::new(psi1.dim/2,  reference_space= :s0, tr )
psi1s0 = Wavefunction::new(system=V1)
psi1s1 = Wavefunction::new(system=V1, :s1)
psi1s1 = psi1s0.s0s1(output => psi1s1)




In [ ]:
block = lambda{
  V0=System::new(ndims,reference_space= :r,bc:free, tr = Transition::new(:r, :s1, :shrink ))
  psi=Wavfunction::new(V0,fill=>)
  phi=psi.rs1
  3.repeat {
    phi.select_s0!
    tmp=phi.s0s1
    phi=tmp
    #alternative solution for the implementation
    phi << phi.s0s1(grow=true)
    }
  #3.repeat{phi.extend_s1!}
  }
block.call
  
block.verbatim
BigDFTV=phi.system



In [ ]:
h= lambda {puts :s1}
puts h.class
a = [1,2,5]
str = a.collect { |e| e.to_s }

In [ ]:
psis=4.times.collect {Wfn::new()}

In [ ]:
psis.each(&:hamapp)